In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os
import tensorflow as tf
import warnings
import tqdm
import random
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
warnings.filterwarnings('ignore')


2024-04-05 13:44:52.621922: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 13:44:52.622068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 13:44:52.804114: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
input_path = []
label = []

for class_name in os.listdir("/kaggle/input/jeel-data/Train"):
    for path in os.listdir("/kaggle/input/jeel-data/Train/"+class_name):
        if class_name == 'AI_Images':
            label.append(0)
        else:
            label.append(1)
        input_path.append(os.path.join("/kaggle/input/jeel-data/Train", class_name, path))
print(input_path[0], label[0])


/kaggle/input/jeel-data/Train/AI_Images/image_561.jpg 0


In [3]:
df = pd.DataFrame()
df['images']= input_path
df['label']= label
df = df.sample(frac=1).reset_index(drop=True)
df.head()


,images,label
0,/kaggle/input/jeel-data/Train/AI_Images/image_...,0
1,/kaggle/input/jeel-data/Train/AI_Images/image_...,0
2,/kaggle/input/jeel-data/Train/Real_Images/imag...,1
3,/kaggle/input/jeel-data/Train/Real_Images/imag...,1
4,/kaggle/input/jeel-data/Train/AI_Images/image_...,0


In [4]:
for i in df['images']:
    if '.jpg' not in i:
        print(i)

In [5]:
df['label'] = df['label'].astype('str')

In [6]:
# input split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)


In [7]:

train_generator = ImageDataGenerator(
    rescale = 1./255,  # normalization of images
    rotation_range = 40, # augmention of images to avoid overfitting
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    )

train_iterator = train_generator.flow_from_dataframe(
    train,
    x_col='images',
    y_col='label',

    target_size=(236,236),
    batch_size=32,
    class_mode='binary',
    
)

val_iterator = val_generator.flow_from_dataframe(
    test,
    x_col='images',
    y_col='label',
    target_size=(236,236),
    batch_size=32,
    class_mode='binary',
    
)


Found 1004 validated image filenames belonging to 2 classes.
Found 251 validated image filenames belonging to 2 classes.


In [8]:
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout

model = Sequential([
                    Conv2D(16, (3,3), activation='relu', input_shape=(236,236,3)),
                    BatchNormalization(),
                    MaxPool2D((2,2)),
                    Conv2D(32, (3,3), activation='relu'),
                    BatchNormalization(),
                    MaxPool2D((2,2)),
                    Conv2D(64, (3,3), activation='relu'),
                    BatchNormalization(),
                    MaxPool2D((2,2)),
                    Conv2D(128, (3,3), activation='relu'),
                    BatchNormalization(),
                    MaxPool2D((2,2)),
                    Flatten(),
                    Dense(128, activation='relu'),
                    Dropout(0.1),
                    Dense(64, activation='relu'),
                    Dropout(0.1),
                    Dense(32, activation='relu'),
                    Dropout(0.1),
                    Dense(1, activation='sigmoid')
])


In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 234, 234, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 234, 234, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 117, 117, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 115, 115, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 115, 115, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 57, 57, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 55, 55, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 55, 55, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 27, 27, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 25, 25, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 25, 25, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,359,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,468,193 (9.42 MB)

 Trainable params: 2,467,713 (9.41 MB)

 Non-trainable params: 480 (1.88 KB)

In [10]:
history = model.fit(train_iterator, epochs=20, validation_data=val_iterator)


Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.5283 - loss: 1.0972 - val_accuracy: 0.4741 - val_loss: 0.9144
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.5359 - loss: 0.9121 - val_accuracy: 0.5896 - val_loss: 0.6819
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.5151 - loss: 0.8585 - val_accuracy: 0.4781 - val_loss: 0.9481
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.5284 - loss: 0.7712 - val_accuracy: 0.4781 - val_loss: 1.2314
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.5581 - loss: 0.7607 - val_accuracy: 0.4781 - val_loss: 0.7114
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.5805 - loss: 0.7319 - val_accuracy: 0.4462 - val_loss: 0.9294
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.5219 - loss: 0.7753 - val_accuracy: 0.4661 - val_loss: 0.7328
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.5720 - loss: 0.7429 - val_accuracy: 0.4861 - val_loss:

In [11]:
images_test = []
labels = []
for test in os.listdir("/kaggle/input/jeel-data/Test/Test_Images"):
  image_path= "/kaggle/input/jeel-data/Test/Test_Images/"+test
  img =load_img(image_path, target_size=(236,236))
  img = np.array(img)
  img = img/255.0
  img = img.reshape(1,236,236,3)
  pred = model.predict(img)
  images_test.append(test.replace(".jpg",""))
  if pred[0][0] > 0.5:
    label= 'Real'
    labels.append(label)
  else:
    label= 'AI'
    labels.append(label)
  print(label)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Real
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Real
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Real
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
AI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
AI
1/1 ━━━━━

In [12]:
test_df = pd.DataFrame()
test_df["Id"] = images_test
test_df["Label"] = labels
test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sort_values(by='Id')
test_df.head()


,Id,Label
76,image_1,AI
290,image_10,AI
297,image_100,AI
52,image_101,Real
477,image_102,Real


In [13]:
test_df.to_csv('submission.csv', index=False)